In [ ]:
import sys
import os
sys.path.append(os.environ['GOTMWORK_ROOT']+'/tools', )
from gotmanalysis import *
%matplotlib inline

In [ ]:
timetag = '20090101-20090131'
casename = 'COREII_Global'
s1data_root = '/Volumes/Qing_Work/work/gotmrun/'+casename
s2data_root = './data/'+casename
fig_root = './fig/'+casename
os.makedirs(s2data_root, exist_ok=True)
os.makedirs(fig_root, exist_ok=True)
update_data = False

In [ ]:
turbmethod_list = ['KPP-CVMix',
                   'KPP-ROMS',
                   'KPPLT-EFACTOR',
                   'KPPLT-ENTR',
                   'KPPLT-RWHGK',
                   'EPBL',
                   'EPBL-LT',
                   'SMC',
                   'SMCLT',
                   'K-EPSILON-SG',
                   'OSMOSIS']
legend_list = ['KPP-CVMix',
               'KPP-ROMS',
               'KPPLT-VR12',
               'KPPLT-LF17',
               'KPPLT-RWHGK16',
               'ePBL',
               'ePBL-LT',
               'SMC-KC94',
               'SMCLT-H15',
               'k-epsilon',
               'OSMOSIS']
nm = len(turbmethod_list)
irow_2col = [1, 2, 0, 1, 2, 3, 3, 4, 4, 5, 5]
icol_2col = [0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1]
labels_2col = ['(b)', '(c)', '(g)', '(h)', '(i)', '(d)', '(j)', '(e)', '(k)','(f)','(l)']

In [ ]:
# get data
for i in np.arange(nm):
    tmname = turbmethod_list[i]
    print(tmname)
    basepath = s1data_root+'/VR1m_DT600s_'+timetag+'/'+tmname
    s2data_name = s2data_root+'/VR1m_DT600s_'+timetag+'/data_mpe_'+tmname+'.npz'
    figname = fig_root+'/VR1m_DT600s_'+timetag+'/fig_pe.png'
    loclist = sorted(os.listdir(basepath))
    if update_data or not os.path.isfile(s2data_name):
        # save data
        pathlist = [basepath+'/'+x+'/gotm_out_s1.nc' for x in loclist]
        godmobj = GOTMOutputDataMap(pathlist)
        gmobj = godmobj.mean_state_timeseries('Epot')
        gmobj.save(s2data_name)
    else:
        # read data
        gmobj = GOTMMap().load(s2data_name)
    if i == 0:
        nloc = len(loclist)
        darr = np.zeros([nm, nloc])
        lon = gmobj.lon
        lat = gmobj.lat
        name = gmobj.name
        units = gmobj.units
    darr[i,:] = gmobj.data


In [ ]:
# create figure
nrow = (nm+2)//2
fig_width = 12
fig_height = 3+2*(nrow-1)

# plot figure
height_ratios = [1]*nrow
height_ratios.append(0.15)
width_ratios = [1, 1, 0.05]
f, axarr = plt.subplots(nrow, 2, sharex='col')
f.set_size_inches(fig_width, fig_height)

for i in np.arange(nm):
    # plot figure
    n = icol_2col[i]
    m = irow_2col[i]
    if i == 0:
        gmdata = darr[i,:]
        gmobj = GOTMMap(data=gmdata, lon=lon, lat=lat, name=name, units=units)
        im1 = gmobj.plot(axis=axarr[m,n], add_colorbar=False, vmax=3e5, vmin=-1e5)
    else:
        gmdata = darr[i,:] - darr[0,:]
        gmobj = GOTMMap(data=gmdata, lon=lon, lat=lat, name=name, units=units)
        im2 = gmobj.plot(axis=axarr[m,n], add_colorbar=False, cmap='RdBu_r', vmax=1e4, vmin=-1e4)
    axarr[m,n].text(0.02, 0.94, labels_2col[i]+' '+legend_list[i], transform=axarr[m,n].transAxes,
                     fontsize=12, color='white', fontweight='bold', va='top')
# standard deviation
dstd = np.std(darr, axis=0)
gmobj = GOTMMap(data=dstd, lon=lon, lat=lat, name=name, units=units)
im0 = gmobj.plot(axis=axarr[0,0], add_colorbar=False, cmap='viridis', vmax=2e4, vmin=0)
axarr[0,0].text(0.02, 0.94, '(a) STD', transform=axarr[0,0].transAxes,
                     fontsize=12, color='white', fontweight='bold', va='top')
    
# reduce margin
plt.tight_layout()

# colorbar
plt.subplots_adjust(right=0.95)
cax0 = plt.axes([0.85, 0.7, 0.1, 0.25])
cax0.set_visible(False)
cb0 = plt.colorbar(im0, ax=cax0)
cb0.formatter.set_powerlimits((-2, 2)) 
cb0.update_ticks()
cax1 = plt.axes([0.85, 0.37, 0.1, 0.25])
cax1.set_visible(False)
cb1 = plt.colorbar(im1, ax=cax1)
cb1.formatter.set_powerlimits((-2, 2)) 
cb1.update_ticks()
cax2 = plt.axes([0.85, 0.04, 0.1, 0.25])
cax2.set_visible(False)
cb2 = plt.colorbar(im2, ax=cax2)
cb2.formatter.set_powerlimits((-2, 2)) 
cb2.update_ticks()

# save figure
plt.savefig(figname, dpi = 300)